In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from summarytools import dfSummary
import warnings
warnings.filterwarnings('ignore')

## 1. Load Data

In [8]:
# Read the datasets
loan_data_train = pd.read_csv('Train.csv')
metadata = pd.read_csv('economic_indicators.csv')
loan_data_test = pd.read_csv('Test.csv')

## 2. EDA

In [10]:
dfSummary(loan_data_train, is_collapsible=False)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,ID[object],1. ID_2666712480322672782. ID_2695752625872672783. ID_2654992484232672784. ID_2856063574202672785. ID_2588882983512672786. ID_2525782779242672787. ID_2540812409712672788. ID_897032603172672789. ID_25778122787226727810. ID_23807126160526727811. other,"1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)1 (0.0%)68,644 (100.0%)","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAD+CAYAAAAtWHdlAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAADLklEQVR4nO3cwY3TYBRG0ecogmFja2QpJUAPKYItvdOFNzCswoIeriVzTgX/4kpPkfxleb1eA4Xb2Q/g/yE2MmIjIzYyYiMjNjJiIyM2MmIjIzYyt2VZvi3L8nb2Q7i+28x8nZnPZz+E63NGyYiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI3MbWZ+zsyfsx/C9S3+xYiKM0pGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyBi8kDF4IeOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEZGbGSsq8hYV5FxRsmIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGYMXMgYvZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzYyFhXkbGuIuOMkhEbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYyBi9kDF7IOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKBmxkREbGesqMtZVZJxRMmIjIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjIzYy1lVkrKvIOKNkxEbG4IWMwQsZZ5SM2MiIjYzYyIiNjNjIiI2M2MiIjYzYyIiNjNjIiI2M2MiIjYzBCxmDFzLOKJllZr7MzKezH8L13fd9/76u6/vZD+H67uu6vj+fz9/btn2c/Riu6ziOt/vMzLZtH4/H49fZD+La/EAgIzYyYiMjNjJiIyM2MmIjIzYyYiMjNjJiIyM2MmIjc5/59/nH2Q/h2o7jeFv2ff/h40kKPgsnY11Fxg8EMmIjIzYyYiMjNjJiIyM2MmIjIzYyfwFj50uCzHpkKgAAAABJRU5ErkJggg=="">",0(0.0%)
2,customer_id[int64],Mean (sd) : 254390.3 (26642.7)min < med < max:145.0 < 255361.0 < 312737.0IQR (CV) : 13323.5 (9.5),"6,540 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACG0lEQVR4nO3dwW6iUBSH8XMHtVQTiDHRnS8wSRcufYh52D7CLHgC38CdCSEQGQkIdDEzSTedKQht/+b7bc3Bs/g2LLjXtW1rwFf37bMXAN5j8tkL4Otwzj2a2eyGR5Rt216G2uc1QoWZ/Y50tVr9CIJg2fcZWZYlzrnnMWIlVPw1C4Jgud/vL2EYFl2H0zT1oyhaxnE8MzNCxbjCMCzW6/WvnuOPgy7zCi9TkECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokECokMDR6HfmhptNgqZppkPvMxRCvSO33GxSVZVf1/X3oih+mlnfM/xHQ6j3pffNJsfjcXk4HObX69Uba7lbEOod6nOzSZIko91oMgRepiCBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBUCGBr1AxmLqup2YWOOf6jJdt217e+pFQMYg8z6dVVT1tt1vP87xOZwqYmWVZljjnnt+KlVAxiLIsJ77vz3e7XbHZbJIus2ma+lEULeM4npkZoWJ8i8Wi8+EXf/zzAIz/huqc883soccf4+MFTdNM0jT1uw6ez2ffzCzPc/90Os0/cv49+74ADxSCZTNjFjMAAAAASUVORK5CYII="">",0(0.0%)
3,country_id[object],1. Kenya,"68,654 (100.0%)",,0(0.0%)
4,tbl_loan_id[int64],Mean (sd) : 263056.3 (39486.7)min < med < max:101323.0 < 260305.5 < 375320.0IQR (CV) : 53020.5 (6.7),"66,520 distinct values","<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAA9hAAAPYQGoP6dpAAACqUlEQVR4nO3cvW7aYBSA4XP4k53Qz0IgsnnpWImBkSV3kIvtmjESSN24gO4IBWEj3FAsmw7N0CFKsDHBh77PfsDDK8v8fEcPh4MAdde49AUAx2hd+gLqSlV9EemUHN8fDoeXKq/nf0eob1BVv9/vPzjnemXmN5vNWlW/E2t1CPVtHedcbzKZvARBsCsyGMex9/T0NFytVgNV3ZR4b+7GbyDUdwRBsBsOh7+KzCRJ0k7TdBSGYbPZbBaKXEQkiqKtqj6KSOHZV1cZOqFWbL/ftzzPuxmPx7u7u7t1kdnFYvFlNpvdh2E4KBO5yPU+dhDqmdze3ha+G6/Xa79s5CJ/Hzum02lvtVp1RIRQcV5lIv+HX+nF1ATfo8IEQoUJhAoTCBUmECpMIFSYQKgwgVBhAqHCBEKFCYQKEwgVJhAqTCBUmECoMIFQYQKhwgRChQmEChMIFSYQKkwgVJhAqDCBUGECocIEQoUJrPS5MlmWtUXEqWqZ8dpuAiTUK3Lqyss6bwIk1CtyysrLum8CJNQrdMI2wNpuAuTDFEwgVJhAqDCBUGECocIEQoUJhAoTCBUmECpMIFSYQKgwgVBhAqHCBEKFCYQKEwgVJhAqTOAf/qiMqvoi0ik5/u7BQkJFJVTV7/f7D865Xpn5jw4WEiqq0nHO9SaTyUsQBIVOwB5zsJBQUakgCM5ysJAPUzCBUGECocIEQoUJhAoTCBUmECpM+

In [17]:
# Check for class imbalance
fig = px.histogram(loan_data_train, 
                   x="target",
                   title='Label Class Balance',
                   color="target",
                   color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(
    width=800,
    height=600,
    showlegend=False,
    title_x=0.5,  # Center the title
    xaxis_title="Label",
    yaxis_title="Count"
)

fig.show()

**Dataset Overview:**

Total Records: 68,654 rows
- Features: 21 columns
- No duplicate entries

**Key Characteristics:**

Loan Types:
- Dominated by Type_1 (89.9%)
- Type_7 (4.1%) and Type_5 (2.2%) are distant seconds


Geographic Coverage:

- All loans are from Kenya (100%)


Loan Details:

- Average Total Amount: 14,837 (with high variation)
- Average Repayment Amount: 15,640
- Duration: Average of 8.5 days, ranging from 1 to 1,096 days
- Disbursement Period: October 2021 to November 2024


Customer Profile:

- Predominantly repeat borrowers (99.2%)
- Only 0.8% new loans


Lender Information:

- Average Lender Funding: 2,546
- Lender Portion: Average 20% of total loan amount
- Only 4 distinct lender IDs in the system


Target Variable:

- Binary classification (0 or 1)
- Highly imbalanced (mean of 0.0 suggests mostly 0s)



**Notable Points:**

- No missing values in most columns
- High variation in loan amounts
- Strong repeat customer base
- Highly imbalanced target variable
- 12.4% missing values in lender_portion_ratio

## 3. Feature Engineering

### 3.1 On train data

In [21]:
# Convert date columns to datetime
loan_data_train['disbursement_date'] = pd.to_datetime(loan_data_train['disbursement_date'])
loan_data_train['due_date'] = pd.to_datetime(loan_data_train['due_date'])

# Extract year and month from dates
loan_data_train['disbursement_year'] = loan_data_train['disbursement_date'].dt.year
loan_data_train['disbursement_month'] = loan_data_train['disbursement_date'].dt.month

# Calculate derived features
loan_data_train['loan_to_repay_ratio'] = loan_data_train['Total_Amount_to_Repay'] / loan_data_train['Total_Amount']
loan_data_train['funded_amount_ratio'] = loan_data_train['Amount_Funded_By_Lender'] / loan_data_train['Total_Amount']
loan_data_train['lender_portion_ratio'] = loan_data_train['Lender_portion_to_be_repaid'] / loan_data_train['Lender_portion_Funded']

# Convert categorical variables to dummy variables
#loan_data = pd.get_dummies(loan_data, columns=['loan_type', 'New_versus_Repeat', 'country_id'])
loan_data_train.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,...,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target,disbursement_year,disbursement_month,loan_to_repay_ratio,funded_amount_ratio,lender_portion_ratio
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,...,Repeat Loan,120.85,0.014305,121.0,0,2022,8,1.000000,0.014305,8458.485726
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,...,Repeat Loan,7768.50,0.300000,7794.0,0,2022,7,1.003244,0.300000,25980.000000
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,...,Repeat Loan,1380.00,0.200000,1428.0,0,2024,9,1.035072,0.200000,7140.000000
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,...,Repeat Loan,2687.40,0.300000,2770.0,0,2022,10,1.030699,0.300000,9233.333333
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,...,Repeat Loan,1369.20,0.300000,1418.0,0,2022,11,1.035933,0.300000,4726.666667


In [23]:
# Process metadata
# Get unique indicators
indicators = metadata['Indicator'].unique()
processed_dfs = {}

# Process each indicator separately
for indicator in indicators:
    indicator_df = metadata[metadata['Indicator'] == indicator].copy()
    
    # Get the latest available year for each country
    latest_year_data = indicator_df.melt(
        id_vars=['Country', 'Indicator'],
        value_vars=[col for col in indicator_df.columns if col.startswith('YR')],
        var_name='Year',
        value_name=indicator.replace(', ', '_').replace(' ', '_').replace('(', '').replace(')', '').lower()
    )
    
    latest_year_data['Year'] = latest_year_data['Year'].str.replace('YR', '').astype(int)
    latest_year_data = latest_year_data.sort_values('Year', ascending=False).groupby('Country').first()
    processed_dfs[indicator] = latest_year_data

# Merge all indicator dataframes
final_metadata = pd.concat([df[df.columns[-1]] for df in processed_dfs.values()], axis=1)
final_metadata = final_metadata.reset_index()
final_metadata

,Country,inflation_consumer_prices_annual_%,official_exchange_rate_lcu_per_us$_period_average,real_interest_rate_%,average_precipitation_in_depth_mm_per_year,deposit_interest_rate_%,lending_interest_rate_%,interest_rate_spread_lending_rate_minus_deposit_rate_%,fossil_fuel_energy_consumption_%_of_total,unemployment_rate
0,Cote d'Ivoire,4.387117,606.569750,6.246748,1348.0,6.490000,5.140000,-1.350000,26.492330,2.403
1,Ghana,38.106966,11.020408,NaN,1187.0,11.416667,NaN,NaN,52.543060,3.079
2,Kenya,7.671396,139.846384,6.546517,630.0,9.167690,13.588502,4.420812,17.379573,5.682


In [24]:
# Merge loan data with metadata based on country
final_train_data = loan_data_train.merge(
    final_metadata,
    left_on='country_id',
    right_on='Country',
    how='left'
)
final_train_data.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,...,Country,inflation_consumer_prices_annual_%,official_exchange_rate_lcu_per_us$_period_average,real_interest_rate_%,average_precipitation_in_depth_mm_per_year,deposit_interest_rate_%,lending_interest_rate_%,interest_rate_spread_lending_rate_minus_deposit_rate_%,fossil_fuel_energy_consumption_%_of_total,unemployment_rate
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682


### 3.2 On test data

In [19]:
# Convert date columns to datetime
loan_data_test['disbursement_date'] = pd.to_datetime(loan_data_test['disbursement_date'])
loan_data_test['due_date'] = pd.to_datetime(loan_data_test['due_date'])

# Extract year and month from dates
loan_data_test['disbursement_year'] = loan_data_test['disbursement_date'].dt.year
loan_data_test['disbursement_month'] = loan_data_test['disbursement_date'].dt.month

# Calculate derived features
loan_data_test['loan_to_repay_ratio'] = loan_data_test['Total_Amount_to_Repay'] / loan_data_test['Total_Amount']
loan_data_test['funded_amount_ratio'] = loan_data_test['Amount_Funded_By_Lender'] / loan_data_test['Total_Amount']
loan_data_test['lender_portion_ratio'] = loan_data_test['Lender_portion_to_be_repaid'] / loan_data_test['Lender_portion_Funded']

# Convert categorical variables to dummy variables
#loan_data = pd.get_dummies(loan_data, columns=['loan_type', 'New_versus_Repeat', 'country_id'])
loan_data_test.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,disbursement_year,disbursement_month,loan_to_repay_ratio,funded_amount_ratio,lender_portion_ratio
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,7,Repeat Loan,575.7,0.300000,597.0,2022,7,1.036477,0.300000,1990.000000
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,7,Repeat Loan,0.0,0.000000,0.0,2022,11,1.007016,0.000000,NaN
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,7,Repeat Loan,207.0,0.025079,208.0,2022,8,1.006058,0.025079,8293.874396
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,7,Repeat Loan,1013.7,0.300000,1014.0,2022,11,1.000000,0.300000,3380.000000
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,7,Repeat Loan,36.0,0.300000,36.0,2022,11,1.000000,0.300000,120.000000


In [26]:
# Merge loan data with metadata based on country
final_test_data = loan_data_test.merge(
    final_metadata,
    left_on='country_id',
    right_on='Country',
    how='left'
)
final_test_data.head()

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,...,Country,inflation_consumer_prices_annual_%,official_exchange_rate_lcu_per_us$_period_average,real_interest_rate_%,average_precipitation_in_depth_mm_per_year,deposit_interest_rate_%,lending_interest_rate_%,interest_rate_spread_lending_rate_minus_deposit_rate_%,fossil_fuel_energy_consumption_%_of_total,unemployment_rate
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,...,Kenya,7.671396,139.846384,6.546517,630.0,9.16769,13.588502,4.420812,17.379573,5.682


In [ ]:
# Drop unnecessary columns
columns_to_drop = ['ID', 'customer_id', 'tbl_loan_id', 'lender_id', 'disbursement_date', 
                  'due_date', 'Country']
final_data = final_data.drop(columns=columns_to_drop)

# Handle missing values
final_data = final_data.fillna(final_data.mean())

# Separate features and target
X = final_data.drop('target', axis=1)
y = final_data['target']

# Print shape of final datasets
print("Features shape:", X.shape)
print("Target shape:", y.shape)

# Print feature names
print("\nFeature names:")
print(X.columns.tolist())

# Basic statistics of numerical columns
print("\nNumerical features statistics:")
print(X.describe())